In [27]:
# %load_ext rpy2.ipython


In [28]:
# %%R

# # Install devtools if you don’t have it already
# install.packages("devtools")

# # Install this package from GitHub
# devtools::install_github("eaegerber/MMLN", force = TRUE)

# # Or install from local source
# # devtools::install("/path/to/MMLN")

In [29]:
# %%R

# library(MMLN)

# # 1. Simulate a small mixed-effects dataset
# set.seed(42)

# sim <- simulate_mixed_mln_data(
#   m       = 10,          # 10 groups
#   n_i     = 10,          # 10 observations per group
#   p       = 3,           # 3 fixed covariates (incl. intercept)
#   d       = 2,           # 3 outcome categories
#   beta    = matrix(c(0.5, -1, 0.2, 0.3, 0.7, -0.4), 3, 2),
#   Sigma   = diag(2),
#   Phi     = 5*diag(2),
#   n_mean = 200
# )

# # 2. Fit a fixed-effects MLN model
# res_f <- FMLN(
#   Y            = sim$Y,
#   X            = sim$X,
#   n_iter       = 1000,
#   burn_in      = 300,
#   thin         = 2,
#   proposal     = "normbeta",
#   verbose      = TRUE
# )

# # 3. Fit a mixed-effects MLN model
# res_m <- MMLN(
#   Y            = sim$Y,
#   X            = sim$X,
#   Z            = sim$Z,
#   n_iter       = 1000,
#   burn_in      = 300,
#   thin         = 2,
#   proposal     = "normbeta",
#   verbose      = TRUE
# )

# # 4. Trace plots & posterior summaries
# beta_chain_array <- simplify2array(res_m$beta_chain)
# trace_stats      <- plot_trace_and_summary(beta_chain_array, "beta")
# trace_stats
# sim$beta
# par(mfrow=c(1,1))

# # 5. Compute model DICs
# ll_chain <- sapply(res_m$w_chain,
#                    function(W) dmnl_loglik(W, sim$Y))
# W_hat   <- alr(compress_counts(sim$Y) / rowSums(sim$Y))
# ll_hat  <- dmnl_loglik(W_hat, sim$Y)
# dic_res <- compute_dic(ll_chain, ll_hat)

# # 6. Posterior predictive simulation and Mahalanobis residuals
# Y_pred_list <- lapply(seq_along(res_m$w_chain), function(i) {
#   sample_posterior_predictive(X = sim$X,
#                               beta = res_m$beta_chain[[i]],
#                               Sigma = res_m$sigma_chain[[i]],
#                               n = sim$n,
#                               Z = sim$Z,
#                               psi = res_m$psi_chain[[i]],
#                               mixed = TRUE,
#                               verbose = FALSE
#   )
# })
# resids <- MDres(sim$Y, Y_pred_list)
# summary(resids)

# # 7. Compare to incorrect model fit (fixed model fit to mixed data; should show some overdispersion)
# Y_pred_list_ovd <- lapply(seq_along(res_f$w_chain), function(i) {
#   sample_posterior_predictive(X = sim$X,
#                               beta = res_f$beta_chain[[i]],
#                               Sigma = res_f$sigma_chain[[i]],
#                               n = sim$n,
#                               mixed = FALSE,
#                               verbose = FALSE
#   )
# })
# resids_ovd <- MDres(sim$Y, Y_pred_list_ovd)
# summary(resids_ovd)

# # 8. Should also show that incorrect model fit has higher DIC
# ll_chain_ovd <- sapply(res_f$w_chain,
#                        function(W) dmnl_loglik(W, sim$Y))
# dic_res_ovd <- compute_dic(ll_chain_ovd, ll_hat)
# dic_res_ovd$DIC > dic_res$DIC

In [30]:
# %%R
# install.packages("Lahman")

In [31]:
# %%R 
# #' Real-Data Example: Pollen Data - MN, DM & MLN Models
# #'
# #' This function reproduces the pollen-data example from Gerber & Craig (2024), fitting three
# #' models to the "pollen" counts (multinomial logit, Dirichlet-multinomial, and
# #' multinomial-logistic-normal), drawing replicate from fitted sampling distributions, and computing
# #' Mahalanobis residuals for each model.
# #'
# #' @param n_iter   Integer; total number of MCMC iterations for the MLN model (default 1000)
# #' @param burn_in  Integer; number of initial MLN iterations to discard (default 400)
# #' @param thin     Integer; MLN thinning interval (default 2)
# #' @param proposal Character; one of `"norm"`, `"beta"`, or `"normbeta"` for the MLN sampler (default `"normbeta"`)
# #' @param P        Integer; number of fitted sampling distribution replicates per model (default 1000)
# #'
# #' @return A list with components:
# #' \describe{
# #'   \item{fit_mlr}{The `MGLMreg` object for the multinomial logit fit.}
# #'   \item{fit_dm}{The `MGLMreg` object for the Dirichlet-multinomial fit.}
# #'   \item{fit_mln}{The list returned by `FMLN()` for the fixed-effects MLN fit.}
# #'   \item{Y_pred_mlr}{List of length P of \eqn{N \times J} count matrices sampled from the MN model.}
# #'   \item{Y_pred_dm}{List of length P of \eqn{N \times J} count matrices sampled from the DM model.}
# #'   \item{Y_pred_mln}{List of length P of \eqn{N \times J} count matrices sampled from the MLN model (using posterior-mean parameters).}
# #'   \item{resids_mlr}{Mahalanobis residuals (`mdres`) for the multinomial logit model.}
# #'   \item{resids_dm}{Mahalanobis residuals (`mdres`) for the Dirichlet-multinomial model.}
# #'   \item{resids_mln}{Mahalanobis residuals (`mdres`) for the MLN model.}
# #' }
# #'
# #' @examples
# #' \dontrun{
# #' # run all three fits & diagnostics
# #' pollen_res <- run_pollen_models(n_iter = 1500, burn_in = 500, thin = 5, P = 500)
# #'
# #' # view KS-tests & QQ-plots
# #' summary(pollen_res$resids_mlr)
# #' summary(pollen_res$resids_dm)
# #' summary(pollen_res$resids_mln)
# #' }
# #'
# #' @importFrom stats rmultinom
# #' @importFrom mvnfast rmvn
# #' @importFrom utils txtProgressBar setTxtProgressBar flush.console data
# #' @importFrom MGLM MGLMreg rdirmn
# #' @export
# run_pollen_models <- function(n_iter   = 1000,
#                               burn_in  = 400,
#                               thin     = 2,
#                               proposal = "normbeta",
#                               P        = 1000) {
#   ## dependencies
#   if (!requireNamespace("MM",   quietly = TRUE)) stop("Install the 'MM' package to load pollen data")
#   if (!requireNamespace("MGLM", quietly = TRUE)) stop("Install the 'MGLM' package for MGLMreg()")

#   ## 1) load data
#   data(pollen, package = "MM", envir = environment())
#   pollen <- as.data.frame(pollen)
#   Y  <- as.matrix(pollen)
#   N  <- nrow(Y)
#   n <- rowSums(Y)
#   J  <- ncol(Y)

#   X <- matrix(1, nrow = N, ncol = 1)  # intercept-only design

#   ## 2) fit M-Logit (multinomial) and Dirichlet-multinomial via MGLM
#   message("? Fitting multinomial-logistic model using MGLMreg()")
#   fit_mlr <- suppressWarnings(MGLMreg(cbind(Pinus, Abies, Quercus, Alnus) ~ 1,
#                      data = as.data.frame(pollen), dist = "MN"))
#   cat("Done: \n")
#   message("? Fitting Dirichlet-multinomial model using MGLMreg()")
#   fit_dm  <- suppressWarnings(MGLMreg(cbind(Pinus, Abies, Quercus, Alnus) ~ 1,
#                      data = as.data.frame(pollen), dist = "DM"))

#   ## 3) fit fixed-effects MLN
#   cat("Done: \n")
#   message("? Fitting multinomial-logistic-normal model using FMLN()")
#   fit_mln <- FMLN(
#     Y              = Y,
#     X              = X,
#     n_iter         = n_iter,
#     burn_in        = burn_in,
#     thin           = thin,
#     proposal       = proposal,
#     verbose        = TRUE
#   )

#   ## 4) prepare predictive replicates
#   cat("Done: \n")
#   message("? Drawing sampling distributions of predicted values from each model")
#   # 4a) MN model: draw P full-dataset replicates
#   probs_mlr <- fit_mlr@fitted
#   Y_pred_mlr <- vector("list", P)
#   for (p_i in seq_len(P)) {
#     M <- t(sapply(seq_len(N),
#                   function(i) rmultinom(1, size = n[i], prob = probs_mlr[i, ])))
#     Y_pred_mlr[[p_i]] <- M
#   }

#   # 4b) DM model: draw P replicates with Dirichlet-multinomial
#   alpha_hat <- exp(fit_dm@coefficients)
#   Y_pred_dm <- vector("list", P)
#   for (p_i in seq_len(P)) {
#     M <- t(sapply(seq_len(N),
#                   function(i) MGLM::rdirmn(n    = 1,
#                                            size = n[i],
#                                            alpha = alpha_hat)))
#     Y_pred_dm[[p_i]] <- M
#   }

#   # 4c) MLN model: use posterior-mean ? and ? to draw P replicates
#   # posterior means:
#   beta_arr  <- simplify2array(fit_mln$beta_chain)
#   Sigma_arr <- simplify2array(fit_mln$sigma_chain)
#   beta_mean  <- apply(beta_arr,  c(1,2), mean)
#   Sigma_mean <- apply(Sigma_arr, c(1,2), mean)

#   Y_pred_mln <- vector("list", P)
#   for (p_i in seq_len(P)) {
#     Y_pred_mln[[p_i]] <- sample_posterior_predictive(
#       X     = X,
#       beta  = beta_mean,
#       Sigma = Sigma_mean,
#       n    = n,
#       mixed = FALSE,
#       verbose = FALSE
#     )
#   }

#   ## 5) compute Mahalanobis-residuals
#   cat("Done: \n")  # ensure we start on a fresh line
#   message("? Computing MDRes for multinomial-logit model")
#   resids_mlr <- MDres(Y, Y_pred_mlr)

#   cat("\nDone: \n")  # ensure we start on a fresh line
#   message("? Computing MDRes for Dirichlet-multinomial model")
#   resids_dm  <- MDres(Y, Y_pred_dm)

#   cat("\nDone: \n")  # ensure we start on a fresh line
#   message("? Computing MDRes for multinomial-logistic-normal model")
#   resids_mln <- MDres(Y, Y_pred_mln)

#   ## return all results
#   output = list(
#     fit_mlr    = fit_mlr,
#     fit_dm     = fit_dm,
#     fit_mln    = fit_mln,
#     Y_pred_mlr = Y_pred_mlr,
#     Y_pred_dm  = Y_pred_dm,
#     Y_pred_mln = Y_pred_mln,
#     resids_mlr = resids_mlr,
#     resids_dm  = resids_dm,
#     resids_mln = resids_mln
#   )

#   return(output)
# }

# #' @importFrom stats model.matrix
# #' @importFrom magrittr %>%
# #' @importFrom dplyr mutate group_by summarise filter arrange recode select left_join slice_max coalesce ungroup across where pull n
# NULL

# if (getRversion() >= "2.15.1") {
#   utils::globalVariables(c(
#     "Batting",     "People",     "Fielding",
#     "lgID",        "playerID",   "yearID",
#     "debut",       "stint",      "InnOuts",
#     "POS",         "pos",        "HR",
#     "BB",          "SO",         "Other",
#     "batsU",       "pos_groupP",
#     "i",           "accept"
#   ))
# }

# #' Prepare Cleaned Lahman Baseball Data for Multinomial Modeling
# #'
# #' This function cleans and processes data from the \pkg{Lahman} database to produce a usable dataset
# #' for modeling MLB player batting outcomes. It returns multinomial response data (`Y`), standardized predictors (`X`),
# #' and a random intercept design matrix (`Z`) for player-level effects.
# #'
# #' The output includes only non-pitcher players with PA > 200 from the American and National Leagues
# #' (post-1960; expansion era), and aggregates data by player-season (grouping across stints and teams).
# #' It collapses batting outcomes into home runs, walks, strikeouts, and other outcomes.
# #' Predictors include physical measurements, age (and age^2), batting side, league, and a grouped fielding position indicator.
# #' Investigating the code, it should not be difficult to adjust the outcomes investigated and/or the predictors.
# #'
# #' @return A list with components:
# #' \describe{
# #'   \item{Y}{An \eqn{N \times J} matrix of batting outcome counts (default: HR, BB, SO, and Other).}
# #'   \item{X}{An \eqn{N \times p} numeric matrix of standardized predictors (default: p = 10).}
# #'   \item{Z}{An \eqn{N \times m} indicator matrix for random intercepts by playerID.}
# #' }
# #'
# #' @examples
# #' \dontrun{
# #' data <- clean_Lahman_data()
# #' str(data$X)
# #' colnames(data$Z)[1:5]
# #' }
# #'
# #' @export
# clean_Lahman_data <- function(){
#   ## dependency
#   if (!requireNamespace("Lahman", quietly = TRUE)) stop("Install the 'Lahman' package to get MLB data")
#   if (!requireNamespace("dplyr", quietly = TRUE)) stop("Install the 'dplyr' package for easy data manipulation")

#   ## 1) load data
#   data(Batting, package = "Lahman", envir = environment())
#   data(People, package = "Lahman", envir = environment())
#   data(Fielding, package = "Lahman", envir = environment())

#   # Let's only care about AL and NL, and post 1900 (Modern Era) or 1960 (Expansion Era)
#   # Fiddle with this for data size
#   year_cutoff <- 1960  # or try 1900 for a larger data set

#   # Compute debut year in AL/NL for each player
#   player_debut <- Batting %>%
#     filter(lgID %in% c("AL","NL")) %>%
#     group_by(playerID) %>%
#     summarise(debut = min(yearID), .groups="drop")

#   # Filter to only those who debuted AFTER the cutoff
#   valid_players <- player_debut %>%
#     filter(debut > year_cutoff) %>%
#     pull(playerID)

#   # Continue with building data set
#   MLB_Batting <- Batting %>%
#     filter(playerID %in% valid_players,
#            lgID     %in% c("AL","NL"),
#            yearID   >  year_cutoff)

#   MLB_People <- People  %>% filter(playerID %in% valid_players)

#   # Now, merge all People information into MLB_Batting
#   MLB_Merged <- merge(MLB_Batting, MLB_People, by = "playerID")

#   # Sort by name, year, stint
#   MLB_Merged <- MLB_Merged %>%
#     arrange(playerID, yearID, stint)

#   # Separate into Info, X, and y data frames
#   # Info (keep some information which may be useful to refer to later; i.e. team or birthCountry)
#   MLB_Info <- MLB_Merged[,c("playerID", "nameFirst", "nameLast", "yearID", "teamID", "lgID", "birthDate", "birthCountry")]

#   # X (predictors; will need to create some and may not use all)
#   MLB_Merged$age <- MLB_Merged$yearID - MLB_Merged$birthYear
#   MLB_X <- MLB_Merged[,c("weight", "height", "age", "bats", "throws", "lgID")]

#   # Want to add Position information (need to use Fielding)
#   # Filter Fielding the same way as above
#   MLB_Fielding <- Fielding %>%
#     filter(playerID %in% valid_players,
#            lgID %in% c("AL","NL"),
#            yearID > year_cutoff)
#   # Replace missing InnOuts with 0
#   MLB_Fielding <- MLB_Fielding %>%
#     mutate(InnOuts = ifelse(is.na(InnOuts), 0, InnOuts))
#   # Get the most played position per player-year-stint (will assume any NA are DH/PH in a sec)
#   Primary_Pos <- MLB_Fielding %>%
#     mutate(InnOuts = coalesce(InnOuts,0)) %>%
#     group_by(playerID, yearID, stint) %>%
#     slice_max(InnOuts, n=1, with_ties=FALSE) %>%
#     ungroup() %>%
#     select(playerID, yearID, stint, POS)
#   # Add to MLB_Merged
#   MLB_Merged_Pos <- MLB_Merged %>%
#     left_join(Primary_Pos, by = c("playerID", "yearID", "stint"))
#   # Finally put it into MLB_X and then update DH/PH for the NA accordingly
#   MLB_X$pos <- MLB_Merged_Pos$POS
#   MLB_X$pos[is.na(MLB_X$pos) & (
#     MLB_Merged_Pos$yearID == 2020 | # COVID
#       (MLB_Merged_Pos$lgID == "AL" & MLB_Merged_Pos$yearID >= 1973) | # AL DH
#       (MLB_Merged_Pos$lgID == "NL" & MLB_Merged_Pos$yearID >= 2023) # NL DH
#   )] <- "DH"
#   MLB_X$pos[is.na(MLB_X$pos)] <- "PH"

#   # y (output; may combine to reduce number of categories, but must sum up to PA)
#   MLB_y <- cbind("X1B" = MLB_Merged$H - rowSums(MLB_Merged[,c("X2B", "X3B", "HR")]),
#                  MLB_Merged[,c("X2B", "X3B", "HR", "BB", "SO", "HBP", "SH", "SF")])
#   MLB_y <- cbind(MLB_y, "OIP" = rowSums(MLB_Merged[,c("AB", "BB", "HBP", "SH", "SF")]) - rowSums(MLB_y))

#   # save the PA (plate appearances; exposure for the Multinomial)
#   PA <- rowSums(MLB_y)

#   # Turn the data frames into useable matrices
#   # Let's focus on the three true outcomes (HR, BB, SO, Other)
#   Y <- as.matrix(cbind(MLB_y[,c("HR", "BB", "SO")], "Other" = PA - rowSums(MLB_y[,c("HR", "BB", "SO")])))

#   # For batting, throws is probably not important, so remove it
#   # Add indicator variables for the categorical columns
#   # Add a quadratic age term
#   # drop unused levels first
#   MLB_X$lgID <- droplevels(factor(MLB_X$lgID))
#   # some unknown batting hands
#   MLB_X$bats <- as.character(MLB_X$bats)
#   MLB_X$bats[is.na(MLB_X$bats)] <- "U" # only necessary for post-1900, not post-1960
#   MLB_X$bats[MLB_X$bats == "B"] <- "S"
#   MLB_X$bats <- factor(MLB_X$bats)
#   MLB_X$bats <- droplevels(factor(MLB_X$bats))
#   # for ease, combine Positions into C, INF, OF, P, DH
#   MLB_X$pos_group <- recode(MLB_X$pos,
#                             "C" = "C",
#                             "1B" = "IF", "2B" = "IF", "SS" = "IF", "3B" = "IF",
#                             "OF" = "OF",
#                             "P" = "P",
#                             "DH" = "DPH", "PH" = "DPH",
#                             .default = "C")
#   MLB_X$pos_group <- factor(MLB_X$pos_group)
#   # scale numeric features
#   MLB_numeric <- cbind(MLB_X[,c("weight", "height", "age")],
#                        "age2" = MLB_X$age^2)

#   X <- as.matrix(cbind(scale(MLB_numeric),
#                        model.matrix(~ bats + lgID + pos_group, data = MLB_X)[,-1]))

#   # Now we need Z for the random effects; these are the player intercepts
#   # based on playerID
#   Z <- model.matrix(~ playerID - 1, data = MLB_Merged)

#   # To make it take less time, and actually fit, Update Y, X, and Z to:
#   # If a player played in the same league in the same year, combine those rows
#   # Ignore pitchers
#   # Keep only rows where PA > 30
#   MLB_Combo <- cbind(MLB_Info[,c("playerID", "yearID", "lgID")],
#                      PA = PA,
#                      Y,
#                      X)
#   MLB_Combo$pos <- MLB_X$pos
#   MLB_Combo <- MLB_Combo %>%
#     filter(pos != "P", PA >= 200) # adjust PA for bigger/smaller data sets

#   MLB_Combo <- MLB_Combo %>%
#     group_by(playerID, yearID, lgID) %>%
#     summarise(
#       across(c(HR, BB, SO, Other, PA), sum),
#       across(where(is.numeric) & !c(HR, BB, SO, Other, PA), mean),
#       .groups = "drop"
#     )

#   MLB_Combo2 <- MLB_Combo %>%
#     group_by(playerID) %>%      # group by player
#     filter(n() > 1) %>%         # keep only players with more than one row
#     ungroup()

#   # New matrices (ignore batsU and pos_groupP, since they are empty now; though don't need to do batsU if post-1960)
#   Y <- as.matrix(MLB_Combo2[, c("HR", "BB", "SO", "Other")])
#   PA <- MLB_Combo2$PA
#   if(year_cutoff == 1900){
#     X <- as.matrix(MLB_Combo2 %>% select(-playerID, -yearID, -lgID, -HR, -BB, -SO, -Other, -PA, -batsU, -pos_groupP))
#   } else{
#     X <- as.matrix(MLB_Combo2 %>% select(-playerID, -yearID, -lgID, -HR, -BB, -SO, -Other, -PA, -pos_groupP))
#   }

#   Z <- model.matrix(~ playerID - 1, data = MLB_Combo2)

#   output = list(
#     Y = Y,
#     X = X,
#     Z = Z,
#     PA = PA
#   )

#   return(output)

# }


In [32]:
# %%R 

# install.packages(c("Lahman", "dplyr", "magrittr"))
# data_out <- clean_Lahman_data()

# str(data_out)
# names(data_out)

# dim(data_out$Y)
# dim(data_out$X)
# dim(data_out$Z)

# head(data_out$Y)
# head(data_out$X)
# head(data_out$Z[,1:5])
# head(data_out$PA)

# summary(data_out$Y)
# summary(data_out$X)


mnk_clean_function

Returns:
    { 
        Y: HR, BB, SO, Other
        X: z-scored: weight (league-specific), height, age, age2 + dummies
        Z: playerID
        PA: plate appearances
        keys: playerID + yearID + lgID
        X_columns: X column names
        players
    }

In [33]:
import pandas as pd
import numpy as np

BATTING_PATH = "mnk_batting.csv"
PEOPLE_PATH = "merged_player_records.csv"
FIELDING_PATH = "mnk_fielding.csv"
LEAGUES = None

batting = pd.read_csv(BATTING_PATH)
people = pd.read_csv(PEOPLE_PATH)
fielding = pd.read_csv(FIELDING_PATH)

if LEAGUES is not None:
    batting = batting[batting["lgID"].isin(LEAGUES)].copy()
    fielding = fielding[fielding["lgID"].isin(LEAGUES)].copy()

/var/folders/yq/z5fxlp7j2z36sz8z8_q3q00r0000gn/T/ipykernel_82628/4152990219.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  batting = pd.read_csv(BATTING_PATH)
/var/folders/yq/z5fxlp7j2z36sz8z8_q3q00r0000gn/T/ipykernel_82628/4152990219.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  fielding = pd.read_csv(FIELDING_PATH)


In [34]:
merged = batting.merge(people, on="playerID", how="inner", suffixes=("", "_ppl"))
display(merged.head(5))

,mnkbID,playerID,mlb_playerID,npb_playerID,kbo_playerID,yearID,stint,lgID,teamID,transition_indicator,...,birthDay,nameLast,nameFirst,nameFull,mlb_weight,npb_weight,kbo_weight,height,bats,throws
0,53,MLB_abadan01_NPB_104799_KBO_,NaN,104799.0,NaN,2000,1,PL,KINTETSU,1,...,25,Abad,Andy,Andy Abad,184,183,184,73,L,L
1,54,MLB_abadan01_NPB_104799_KBO_,abadan01,NaN,NaN,2001,1,AL,OAK,2,...,25,Abad,Andy,Andy Abad,184,183,184,73,L,L
2,55,MLB_abadan01_NPB_104799_KBO_,abadan01,NaN,NaN,2003,1,AL,BOS,2,...,25,Abad,Andy,Andy Abad,184,183,184,73,L,L
3,56,MLB_abadan01_NPB_104799_KBO_,abadan01,NaN,NaN,2006,1,NL,CIN,3,...,25,Abad,Andy,Andy Abad,184,183,184,73,L,L
4,139,MLB_abbotky01_NPB_104266_KBO_,abbotky01,NaN,NaN,1991,1,AL,CAL,1,...,18,Abbott,Kyle,Kyle Abbott,200,201,200,76,L,L


In [35]:
# primary POS per player-year-stint 
fld = fielding.copy()
fld["G"] = pd.to_numeric(fld.get("G", 0), errors="coerce").fillna(0.0)

# retrieve stint-level primary POS, season-level primary POS, career-level primary POS
stint_pos = (
    fld.sort_values(["playerID","yearID","stint","G"], ascending=[True,True,True,False])
       .drop_duplicates(["playerID","yearID","stint"])
       [["playerID","yearID","stint","POS"]]
       .rename(columns={"POS":"POS_stint"}))

season_pos = (
    fld.groupby(["playerID","yearID","lgID","POS"], as_index=False)["G"].sum()
       .sort_values(["playerID","yearID","lgID","G"], ascending=[True,True,True,False])
       .drop_duplicates(["playerID","yearID","lgID"])
       [["playerID","yearID","lgID","POS"]]
       .rename(columns={"POS":"POS_season"}))

career_pos = (
    fld.groupby(["playerID","POS"], as_index=False)["G"].sum()
       .sort_values(["playerID","G"], ascending=[True,False])
       .drop_duplicates(["playerID"])
       [["playerID","POS"]]
       .rename(columns={"POS":"POS_career"}))

merged = (
    merged.sort_values(["playerID","yearID","stint"])
          .merge(stint_pos,  on=["playerID","yearID","stint"], how="left")
          .merge(season_pos, on=["playerID","yearID","lgID"], how="left")
          .merge(career_pos, on="playerID", how="left"))

merged["POS_primary"] = merged["POS_stint"]
merged["POS_primary"] = merged["POS_primary"].fillna(merged["POS_season"])
merged["POS_primary"] = merged["POS_primary"].fillna(merged["POS_career"])

pos_missing = int(merged["POS_primary"].isna().sum())
print(pos_missing)
display(merged[["playerID","yearID","stint","lgID","teamID","POS_primary"]].head(20))

0


,playerID,yearID,stint,lgID,teamID,POS_primary
0,MLB__NPB_102080_KBO_96480,1963,1,PL,TOEI,C
1,MLB__NPB_102080_KBO_96480,1964,1,PL,TOEI,C
2,MLB__NPB_102080_KBO_96480,1965,1,PL,TOEI,C
3,MLB__NPB_102080_KBO_96480,1966,1,PL,TOEI,OF
4,MLB__NPB_102080_KBO_96480,1967,1,PL,TOEI,OF
5,MLB__NPB_102080_KBO_96480,1968,1,PL,TOEI,OF
6,MLB__NPB_102080_KBO_96480,1969,1,PL,TOEI,OF
7,MLB__NPB_102080_KBO_96480,1970,1,PL,TOEI,OF
8,MLB__NPB_102080_KBO_96480,1971,1,PL,TOEI,OF
9,MLB__NPB_102080_KBO_96480,1972,1,PL,TOEI,OF


In [36]:
# age, weight, bats, pos (with DH/PH rules)

# age
merged["age"] = (merged["yearID"] - merged["birthYear"]).astype(float)

# league-specific weight
for weight in ["mlb_weight","npb_weight","kbo_weight"]:
    if weight not in merged.columns:
        merged[weight] = np.nan

def choose_weight(row):
    lg = row["lgID"]
    w_mlb = row.get("mlb_weight", np.nan)
    w_npb = row.get("npb_weight", np.nan)
    w_kbo = row.get("kbo_weight", np.nan)
    mapping = {"AL": w_mlb, "NL": w_mlb, "CL": w_npb, "PL": w_npb, "KBO": w_kbo}
    pick = mapping.get(lg, np.nan)
    if pd.notna(pick) and pick != 0:
        return pick
    for v in (w_mlb, w_npb, w_kbo):
        if pd.notna(v) and v != 0:
            return v
    return np.nan

merged["weight"] = merged.apply(choose_weight, axis=1)

# normalize bats B -> S
bats = merged.get("bats", pd.Series(index=merged.index, dtype=object)).astype(str).str.upper()
bats = bats.replace({"B":"S", "NAN":np.nan}).fillna("U")
merged["bats_norm"] = bats

# POS final fill:
#   AL: DH since 1973
#   NL: DH since 2022
#   PL: DH since 1975
#   CL: no DH so PH
#   KBO: DH used
if pos_missing != 0:
    POS_primary = merged["POS_primary"].copy()
    mask_na = POS_primary.isna()
    dh_AL = (merged["lgID"].eq("AL") & (merged["yearID"] >= 1973))
    dh_NL = (merged["lgID"].eq("NL") & (merged["yearID"] >= 2022))
    dh_PL = (merged["lgID"].eq("PL") & (merged["yearID"] >= 1975))
    dh_KBO = (merged["lgID"].eq("KBO"))
    mask_dh = mask_na & (dh_AL | dh_NL | dh_PL | dh_KBO)

    # fill DH where DH leagues apply, else PH
    POS_primary = POS_primary.where(~mask_dh, "DH")
    POS_primary = POS_primary.fillna("PH")
    merged["POS_primary"] = POS_primary
    display(merged[["playerID","yearID","lgID","age","height","weight","bats_norm","POS_primary"]].head(20))
    
display(merged[["playerID","yearID","lgID","age","height","weight","bats_norm","POS_primary"]].head(20))

,playerID,yearID,lgID,age,height,weight,bats_norm,POS_primary
0,MLB__NPB_102080_KBO_96480,1963,PL,20.0,69,181,R,C
1,MLB__NPB_102080_KBO_96480,1964,PL,21.0,69,181,R,C
2,MLB__NPB_102080_KBO_96480,1965,PL,22.0,69,181,R,C
3,MLB__NPB_102080_KBO_96480,1966,PL,23.0,69,181,R,OF
4,MLB__NPB_102080_KBO_96480,1967,PL,24.0,69,181,R,OF
5,MLB__NPB_102080_KBO_96480,1968,PL,25.0,69,181,R,OF
6,MLB__NPB_102080_KBO_96480,1969,PL,26.0,69,181,R,OF
7,MLB__NPB_102080_KBO_96480,1970,PL,27.0,69,181,R,OF
8,MLB__NPB_102080_KBO_96480,1971,PL,28.0,69,181,R,OF
9,MLB__NPB_102080_KBO_96480,1972,PL,29.0,69,181,R,OF


In [37]:
# Build PA at row level; prefer provided PA, else AB+BB+HBP+SH+SF
df = merged.copy()

# Ensure HR/BB/SO/AB/HBP/SH/SF are numeric
if "PA" in df.columns:
    df["PA_row"] = pd.to_numeric(df["PA"], errors="coerce")
else:
    # ONLY build from stats if they actually exist
    comp_cols = [c for c in ["AB","BB","HBP","SH","SF"] if c in df.columns]
    if comp_cols:
        for c in comp_cols:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df["PA_row"] = df[comp_cols].sum(axis=1, min_count=1)
    else:
        df["PA_row"] = np.nan

# Row-level scaffold (no 'Other' yet)
row_outcomes = df[["playerID","yearID","lgID","HR","BB","SO","PA_row"]].copy()
row_outcomes.rename(columns={"PA_row":"PA"}, inplace=True)

display(row_outcomes.head(10))


,playerID,yearID,lgID,HR,BB,SO,PA
0,MLB__NPB_102080_KBO_96480,1963,PL,0,0,4.0,19.0
1,MLB__NPB_102080_KBO_96480,1964,PL,6,8,34.0,264.0
2,MLB__NPB_102080_KBO_96480,1965,PL,14,11,28.0,378.0
3,MLB__NPB_102080_KBO_96480,1966,PL,4,12,30.0,382.0
4,MLB__NPB_102080_KBO_96480,1967,PL,10,15,24.0,431.0
5,MLB__NPB_102080_KBO_96480,1968,PL,15,9,21.0,397.0
6,MLB__NPB_102080_KBO_96480,1969,PL,12,13,31.0,472.0
7,MLB__NPB_102080_KBO_96480,1970,PL,18,36,39.0,540.0
8,MLB__NPB_102080_KBO_96480,1971,PL,11,26,37.0,448.0
9,MLB__NPB_102080_KBO_96480,1972,PL,19,35,32.0,529.0


In [38]:
agg_y = (row_outcomes
         .groupby(["playerID","yearID","lgID"], as_index=False)
         .agg(PA=("PA","sum"), HR=("HR","sum"), BB=("BB","sum"), SO=("SO","sum")))

merged["pos"] = merged["POS_primary"]

# season-level predictors 
tmp = merged[["playerID","yearID","lgID","pos","bats_norm","height","weight","age"]].copy()
agg_num = (tmp.groupby(["playerID","yearID","lgID"], as_index=False).agg(height=("height","mean"), weight=("weight","mean"), age=("age","mean")))
mode_fn = lambda s: s.mode(dropna=True).iat[0] if not s.mode(dropna=True).empty else np.nan
agg_cat = (tmp.groupby(["playerID","yearID","lgID"], as_index=False).agg(bats_norm=("bats_norm", mode_fn), pos=("pos", mode_fn)))

# combine outcomes + predictors
combo = (agg_y.merge(agg_num, on=["playerID","yearID","lgID"], how="left").merge(agg_cat, on=["playerID","yearID","lgID"], how="left"))

# compute Other from season totals so PA == HR+BB+SO+Other
for c in ["PA","HR","BB","SO"]:
    combo[c] = pd.to_numeric(combo[c], errors="coerce").fillna(0.0)

combo["Other"] = (combo["PA"] - combo["HR"] - combo["BB"] - combo["SO"]).astype(float)
combo.loc[combo["Other"] < 0, "Other"] = 0.0

# combo = combo[combo["pos"] != "P"].reset_index(drop=True)

sum_check = np.isclose(
    combo[["HR","BB","SO","Other"]].sum(axis=1).to_numpy(),
    combo["PA"].to_numpy())

print(int((~sum_check).sum()))
display(combo[["playerID","yearID","lgID","PA","HR","BB","SO","Other"]].head(200))

30


,playerID,yearID,lgID,PA,HR,BB,SO,Other
0,MLB__NPB_102080_KBO_96480,1963,PL,19.0,0,0,4.0,15.0
1,MLB__NPB_102080_KBO_96480,1964,PL,264.0,6,8,34.0,216.0
2,MLB__NPB_102080_KBO_96480,1965,PL,378.0,14,11,28.0,325.0
3,MLB__NPB_102080_KBO_96480,1966,PL,382.0,4,12,30.0,336.0
4,MLB__NPB_102080_KBO_96480,1967,PL,431.0,10,15,24.0,382.0
...,...,...,...,...,...,...,...,...
195,MLB__NPB_103809_KBO_94408,2000,KBO,14.0,0,1,2.0,11.0
196,MLB__NPB_103970_KBO_73229,1990,PL,0.0,0,0,0.0,0.0
197,MLB__NPB_103970_KBO_73229,1991,PL,0.0,0,0,0.0,0.0
198,MLB__NPB_103970_KBO_73229,1992,PL,0.0,0,0,0.0,0.0


In [39]:
diff = combo["PA"] - (combo["HR"] + combo["BB"] + combo["SO"])
print("rows with PA < HR+BB+SO = ", int((diff < 0).sum()))
print("rows with PA > HR+BB+SO = ", int((diff > 0).sum()))
display(combo.loc[diff < 0, ["playerID","yearID","lgID","PA","HR","BB","SO"]].head(10))


rows with PA < HR+BB+SO =  30
rows with PA > HR+BB+SO =  8361


,playerID,yearID,lgID,PA,HR,BB,SO
1090,MLB_baumeji01_NPB_102035_KBO_,1949,AL,0.0,0,2,1.0
1205,MLB_bertore01_NPB_102110_KBO_,1953,AL,0.0,0,0,1.0
2876,MLB_dobyla01_NPB_101920_KBO_,1947,AL,0.0,0,1,11.0
2877,MLB_dobyla01_NPB_101920_KBO_,1948,AL,0.0,14,54,77.0
2878,MLB_dobyla01_NPB_101920_KBO_,1949,AL,0.0,24,91,90.0
2879,MLB_dobyla01_NPB_101920_KBO_,1950,AL,0.0,25,98,71.0
2880,MLB_dobyla01_NPB_101920_KBO_,1951,AL,0.0,20,101,81.0
2881,MLB_dobyla01_NPB_101920_KBO_,1952,AL,0.0,32,90,111.0
2882,MLB_dobyla01_NPB_101920_KBO_,1953,AL,0.0,29,96,121.0
3353,MLB_foytapa01_NPB_102189_KBO_,1953,AL,0.0,0,0,1.0


In [40]:
# fix PA value 
# If PA >= HR+BB+SO then set Other = PA − (HR+BB+SO)
# If PA < HR+BB+SO or PA is NaN then set PA = HR+BB+SO and Other = 0

for c in ["PA","HR","BB","SO"]:
    combo[c] = pd.to_numeric(combo[c], errors="coerce")

sum_outcomes = combo[["HR","BB","SO"]].sum(axis=1)
diff = combo["PA"] - sum_outcomes

# Rows to fix: PA is NaN or PA < HR+BB+SO
fix = combo["PA"].isna() | (diff < 0)

combo.loc[fix, "PA"] = sum_outcomes[fix].astype(float)

# compute Other from (possibly updated) PA
combo["Other"] = (combo["PA"] - sum_outcomes).clip(lower=0).astype(float)

In [41]:

sum_check = np.isclose(
    combo[["HR","BB","SO","Other"]].sum(axis=1).to_numpy(),
    combo["PA"].to_numpy())

print(int((~sum_check).sum()))


0


In [42]:
# Build X
# age2 
combo["age2"] = combo["age"] ** 2

# position groups (C baseline; keep DPH/IF/OF dummies)
def pos_group_fn(p):
    if p == "C":
        return "C"
    if p in ("1B","2B","SS","3B"):
        return "IF"
    if p == "OF":
        return "OF"
    if p == "P":
        return "P"
    if p in ("DH","PH"):
        return "DPH"
    return "C"

if "pos_group" not in combo.columns:
    combo["pos_group"] = combo["pos"].map(pos_group_fn)

# z-score ((x1 - mean) / std) for weight, height, age, age2
X_num = combo[["weight","height","age","age2"]].astype(float)
X_num = (X_num - X_num.mean()) / X_num.std(ddof=0)

# bats: set factor baselines
#   bats baseline: L (dummies for R and S)
#   pos group baseline (so we get DPH, IF, OF)
#   lgID baseline = first in preferred order below

combo["bats_norm"] = pd.Categorical(combo["bats_norm"].fillna("U"), categories=["L","R","S","U"], ordered=False)

# prefer MLB baselines if present
preferred_lg_order = ["AL","NL","CL","PL","KBO","FL","JBL","UA","UPL"]
present = [lg for lg in preferred_lg_order if lg in set(combo["lgID"].dropna())]
others = [lg for lg in combo["lgID"].dropna().unique() if lg not in present]
lg_levels = present + list(others) if present else sorted(combo["lgID"].dropna().unique().tolist())
combo["lgID"] = pd.Categorical(combo["lgID"], categories=lg_levels, ordered=False)


combo["pos_group"] = pd.Categorical(
    combo["pos_group"],
    categories=["C","DPH","IF","OF"],
    ordered=False)

# factor dummies with one level dropped per factor
X_cat = pd.get_dummies(combo[["bats_norm","lgID","pos_group"]],drop_first=True)
if "bats_norm_U" in X_cat.columns:
    X_cat = X_cat.drop(columns=["bats_norm_U"])
# Final X = scaled numeric + factor dummies
X_df = pd.concat([X_num, X_cat], axis=1)

X_columns = X_df.columns.tolist()
display(X_df.head(20))


,weight,height,age,age2,bats_norm_R,bats_norm_S,lgID_NL,lgID_CL,lgID_PL,lgID_KBO,pos_group_DPH,pos_group_IF,pos_group_OF
0,-0.854643,-1.908455,-2.127221,-1.810067,True,False,False,False,True,False,False,False,False
1,-0.854643,-1.908455,-1.888178,-1.646115,True,False,False,False,True,False,False,False,False
2,-0.854643,-1.908455,-1.649136,-1.474165,True,False,False,False,True,False,False,False,False
3,-0.854643,-1.908455,-1.410094,-1.294217,True,False,False,False,True,False,False,False,True
4,-0.854643,-1.908455,-1.171051,-1.106272,True,False,False,False,True,False,False,False,True
5,-0.854643,-1.908455,-0.932009,-0.910329,True,False,False,False,True,False,False,False,True
6,-0.854643,-1.908455,-0.692967,-0.706389,True,False,False,False,True,False,False,False,True
7,-0.854643,-1.908455,-0.453924,-0.494450,True,False,False,False,True,False,False,False,True
8,-0.854643,-1.908455,-0.214882,-0.274514,True,False,False,False,True,False,False,False,True
9,-0.854643,-1.908455,0.024160,-0.046581,True,False,False,False,True,False,False,False,True


In [43]:
# Build Y: outcome matrix (HR, BB, SO, Other)
Y = combo[["HR","BB","SO","Other"]].to_numpy(dtype=np.float64)
PA_out = combo["PA"].to_numpy(dtype=np.float64)

In [44]:
len_check = (Y.shape[0] == PA_out.shape[0])
sum_check2 = np.allclose(Y.sum(axis=1), PA_out, atol=1e-8)
non_neg_check = np.all((Y >= 0).ravel()) and np.all(PA_out >= 0)

print(len_check)
print(sum_check2)
print(non_neg_check)

True
True
True


In [45]:
display(Y[:20])

array([[  0.,   0.,   4.,  15.],
       [  6.,   8.,  34., 216.],
       [ 14.,  11.,  28., 325.],
       [  4.,  12.,  30., 336.],
       [ 10.,  15.,  24., 382.],
       [ 15.,   9.,  21., 352.],
       [ 12.,  13.,  31., 416.],
       [ 18.,  36.,  39., 447.],
       [ 11.,  26.,  37., 374.],
       [ 19.,  35.,  32., 443.],
       [  6.,  16.,  22., 268.],
       [ 15.,  22.,  19., 401.],
       [ 16.,  17.,  18., 352.],
       [ 17.,  23.,  33., 425.],
       [ 16.,  30.,  33., 411.],
       [  3.,   7.,  12., 159.],
       [ 18.,  21.,  25., 379.],
       [  5.,   7.,  17., 153.],
       [  4.,  14.,  12., 184.],
       [ 19.,  42.,  17., 214.]])

In [46]:
# Build Z: random-effects matrix -- ignore for now

# 1) prebuilt multinomial (binary, probability) logistic model -- vignette  
# 2) NN
# R to fit the model -- one with / without Z;


# age, height, weight z_score scale using full data set (still merged_player_records), not league-specific
# Summary report - intro, building database, data prep & ML, future works (depends on how much done)
# Turn into a package; one package containing datasets

Multinomia Logistic Regression model - A vignette to be made

Steps:
1) Find player-rows where the next year is in a different league
2) Save those rows
3) Fit a multinomial logistic regression on the remaining rows
4) Predict the YR+1 rows and compare to actual HR/BB/SO/Other
5) Graph -- past study: may overpredict performance of npb, underperformance of mlb 
         -- IF AND ONLY IF: two / three bionomial e.g. HR vs. not homeruns (counts of HR and not HR)

In [47]:
# combo, X_df from data prep stage above
combo = combo.reset_index(drop=True)
X_df = X_df.reset_index(drop=True)

# order/columns to match with Y (outcome matrix)
class_names = ["HR", "BB", "SO", "Other"]
if Y.shape[1] == len(class_names):
    print("@@@@")

@@@@


In [48]:

# identify cross-league targets
# “targets” are rows for year t where player also played year t-1 and the league changed from t-1 -> t (that t row is what we predict)

ids = combo[["playerID","yearID","lgID"]].copy()
ids["row"] = ids.index
ids = ids.sort_values(["playerID","yearID"])

# previous year/league (within each player)
ids["prev_year"] = ids.groupby("playerID")["yearID"].shift(1)
ids["prev_lg"] = ids.groupby("playerID")["lgID"].shift(1)

# target rows: consecutive seasons AND league changed
targets = ids[(ids["prev_year"] == ids["yearID"] - 1) & (ids["lgID"] != ids["prev_lg"])].copy()

len(targets) # saving these
display(targets.head(10)[["row","playerID","prev_year","prev_lg","yearID","lgID"]])


,row,playerID,prev_year,prev_lg,yearID,lgID
19,19,MLB__NPB_102080_KBO_96480,1981.0,PL,1982,KBO
26,26,MLB__NPB_102608_KBO_83999,1972.0,CL,1973,PL
30,30,MLB__NPB_102608_KBO_83999,1976.0,PL,1977,CL
54,54,MLB__NPB_102687_KBO_84999,1991.0,CL,1992,PL
60,60,MLB__NPB_102771_KBO_86303,1976.0,CL,1977,PL
68,68,MLB__NPB_102771_KBO_86303,1984.0,PL,1985,CL
79,79,MLB__NPB_102864_KBO_80057,1983.0,PL,1984,KBO
86,86,MLB__NPB_102893_KBO_83800,1982.0,CL,1983,KBO
96,96,MLB__NPB_103035_KBO_84573,1983.0,PL,1984,KBO
108,108,MLB__NPB_103094_KBO_88274,1982.0,PL,1983,CL


In [49]:
# indices for saved rows (yr+1 different league)
target_idx = targets["row"].to_numpy()
train_mask = ~combo.index.isin(target_idx)

# train arrays
X_train = X_df.loc[train_mask].to_numpy(dtype=float)
Y_train = Y[train_mask, :]

# target arrays (to be predicted)
X_tgt = X_df.loc[target_idx].to_numpy(dtype=float)
Y_tgt = Y[target_idx, :]
PA_tgt = PA_out[target_idx]

print("Train rows:", X_train.shape[0])
print("Target rows:", X_tgt.shape[0])

Train rows: 9524
Target rows: 2052


Scikit-learn requires training data in:
  - X: one feature vector per row
  - Y: one single class ID per row e.g. 0 = HR, 1 = BB, 2 = SO, 3 = Other 
Current: grouped counts Y_ij = number of PAs for row i in class j.
Solution: create a long dataset with len(Y_counts.shape[1]) = 4 rows for each season and give each row a sample weight equal to the count


Visualization - dummy example (for myself): 

BEFORE
- X_mat (2 seasons x 3 features) 
  =  [[  1.0   0.5  -1.2],   # row 0: height, weight, age
      [  0.3   -0.7  0.0]]   # row 1: height, weight, age

- Y_counts (2 seasons x 4 outcomes)
  =  [[2,  1,  0,  3],      # row 0: HR=2, BB=1, SO=0, Other=3
      [0,  4,  1,  5]]      # row 1: HR=0, BB=4, SO=1, Other=5

DURING
- y_long (2 seasons * 4 outcomes) (class that I am modeling on this replicated row)
 = [0, 1, 2, 3, 0, 1, 2, 3]      #  indices {HR:0, BB:1, SO:2, Other:3}

- w_long (2 seasons * 4 outcomes) (num of times repeated the row if expanded per-PA)
 = [Y[0,0], Y[0,1], Y[0,2], Y[0,3], Y[1,0], Y[1,1], Y[1,2], Y[1,3]]
 = [2, 1, 0, 3, 0, 4, 1, 5]

- x_long (2 seasons * 4 outcomes * 3 features) (features repeated 4 times for each original row)
 =
[[ 1.0  0.5  -1.2],   # row0, HR
 [ 1.0  0.5  -1.2],   # row0, BB
 [ 1.0  0.5  -1.2],   # row0, SO
 [ 1.0  0.5  -1.2],   # row0, Other
 [ 0.3  -0.7  0.0],   # row1, HR
 [ 0.3  -0.7  0.0],   # row1, BB
 [ 0.3  -0.7  0.0],   # row1, SO
 [ 0.3  -0.7  0.0]]   # row1, Other

 *** Drop zero-weight rows
keep = [ True, True, False, True, False, True, True, True]

AFTER
- x_long 
 = 
[[ 1.0  0.5 -1.2],   # row0, HR     (weight 2)
 [ 1.0  0.5 -1.2],   # row0, BB     (weight 1)
 [ 1.0  0.5 -1.2],   # row0, Other  (weight 3)
 [ 0.3 -0.7  0.0],   # row1, BB     (weight 4)
 [ 0.3 -0.7  0.0],   # row1, SO     (weight 1)
 [ 0.3 -0.7  0.0]]   # row1, Other  (weight 5)

- y_long 
 = [0, 1, (2), 3, (0), 1, 2, 3]
 / [2, 1, (0), 3, (0), 4, 1, 5]
 ---------------------------
 = [0, 1,      3,      1, 2, 3] = [0, 1, 3, 1, 2, 3]

- w_long
 [2., 1., 3., 4., 1., 5.]


## subset the data, POC -> add requirements to reduce the data size; e.g. 1960+
## 

In [50]:
from sklearn.linear_model import LogisticRegression

def build_long(X_mat, Y_counts):
    """
    returns:
    - X_long : repeated feature rows (N*J, p)
    - y_long : class labels 0..J-1 (length N*J)
    - w_long : sample weights = counts (length N*J)
    *keep : drop zero-weight rows; contribute nothing to the likelihood
    """
    N, p = X_mat.shape
    J = Y_counts.shape[1]
    y_long = np.tile(np.arange(J, dtype=int), N)
    w_long = Y_counts.reshape(-1).astype(float)
    X_long = np.repeat(X_mat, J, axis=0) 

    keep = w_long > 0
    return X_long[keep], y_long[keep], w_long[keep]

X_long_tr, y_long_tr, w_long_tr = build_long(X_train, Y_train)

